In [1]:
def revcomp(seq):
	seq=seq.replace('A','X')
	seq=seq.replace('T','A')
	seq=seq.replace('X','T')
	seq=seq.replace('C','X')
	seq=seq.replace('G','C')
	seq=seq.replace('X','G')
	return seq[::-1]
def distance(a, b):
    return sum(map(lambda xy: 0 if xy[0] == xy[1] else 1, zip(a, b)))
def mismatch(k, a, b):
    return distance(a, b) < k

import time
import logging
import sys
import gzip

import numpy as np
import json

seqnum = 1
qualnum = 3
blocknum = 4
lognum = 500000

In [11]:
samples = ['ky10', 'ky11']

In [4]:
for sample_index, x in enumerate(samples):
    
    infiles = ['data/'+x+'/R1_trim.fastq.gz', 'data/'+x+'/R2_trim.fastq.gz']
    readdir = ['f', 'r']

    starttime = time.time()
    readno = 0
    
    lognum = 1000000
    barcode_len = 7

    barcodes1 = {}
    barcodes2 = {}
    dropped = 0
    dropped_short = 0
    dropped_prime = 0
    dropped_bar = 0
    
    #primer mismatches must be less than:
    fprime_mismatches = 4
    rprime_mismatches = 4
    
    fprime1 = 'GCAGTGAAAGAGTTCTTCGGTTCCA'
    rprime1 = 'TCTGACCCGTGAACGTGCA'
    target1 = 'GCAGTGAAAGAGTTCTTCGGTTCCAGCCAGCTGTCTCAGTTTATGGACCAGAACAACCCGCTGTCTGAGATTACGCACAAACGTCGTATCTCCGCACTCGGCCCAGGCGGTCTGACCCGTGAACGTGCA'
    seq_len1 = (2 * barcode_len) + len(target1)
    
    fprime2 = 'CGAAGTTCGAGACGTACACCCG'
    rprime2 = 'CGAATACGGCTTCCTTGAGACTCC'
    target2 = 'CGAAGTTCGAGACGTACACCCGACTCACTACGGTCGCGTATGTCCAATCGAAACCCCTGAAGGTCCGAACATCGGTCTGATCAACTCTCTGTCCGTGTACGCACAGACTAACGAATACGGCTTCCTTGAGACTCC'
    seq_len2 = (2 * barcode_len) + len(target2)

    for filenumber, f in enumerate(infiles):
        rc = False
        if readdir[filenumber] == 'r':
            rc = True

        linenum = 0
        fin = gzip.open(f,'r')

        for line in fin:
            line = line.decode("utf-8")
            
            if linenum%lognum == 0:
                print ('processed line '+str(linenum)+'. time: '+str(time.time()-starttime))

            if linenum%blocknum == seqnum:
                readno = readno + 1
                sequence = line.rstrip()
    
                if rc:
                    if mismatch(rprime_mismatches, revcomp(rprime1), sequence[barcode_len:barcode_len + len(rprime1)]):
                        sequence = sequence[:seq_len1]
                    if mismatch(rprime_mismatches, revcomp(rprime2), sequence[barcode_len:barcode_len + len(rprime2)]):
                        sequence = sequence[:seq_len2]
                        
                    sequence = revcomp(sequence)
                else:
                    #hack for shitty trimming
                    if mismatch(fprime_mismatches, fprime1, sequence[barcode_len:barcode_len + len(fprime1)]):
                        sequence = sequence[:seq_len1]
                    if mismatch(fprime_mismatches, fprime2, sequence[barcode_len:barcode_len + len(fprime2)]):
                        sequence = sequence[:seq_len2]

                if (len(sequence) != seq_len1) and (len(sequence) != seq_len2):
                    #print(sequence)
                    linenum = linenum + 1
                    dropped += 1
                    dropped_short += 1
                    #dropped_lines.write(line+"\t"+"incomplete sequence\n")
                    continue
                    
                #hack for ba trimming
                #if (len(sequence) == seq_len2 + 2):
                #    sequence = sequence[0:-2]

                #if both primers found in sequence
                if mismatch(fprime_mismatches, fprime1, sequence[barcode_len:barcode_len + len(fprime1)]) and mismatch(rprime_mismatches, rprime1, sequence[len(sequence) - barcode_len - len(rprime1):len(sequence) - barcode_len]):
                    barcode = sequence[0:barcode_len] + sequence[len(sequence)-barcode_len:len(sequence)]

                    #sequence without barcodes but including both primers
                    subseq = sequence[barcode_len:len(sequence)-barcode_len]

                    #check if barcode is complete, otherwise drop
                    if not 'N' in barcode:

                        #check if barcode has been found already
                        if barcode in barcodes1:
                            barcodes1[barcode][0] += 1
                            barcodes1[barcode][1].append(subseq)
                        else:
                            barcodes1[barcode] = [1, [subseq]]
                    else:
                        dropped = dropped + 1
                        dropped_bar += 1
                        #dropped_lines.write(line+"\t"+"incomplete barcode\n")
                
                elif mismatch(fprime_mismatches, fprime2, sequence[barcode_len:barcode_len + len(fprime2)]) and mismatch(rprime_mismatches, rprime2, sequence[len(sequence) - barcode_len - len(rprime2):len(sequence) - barcode_len]):
                    barcode = sequence[0:barcode_len] + sequence[len(sequence)-barcode_len:len(sequence)]

                    #sequence without barcodes but including both primers
                    subseq = sequence[barcode_len:len(sequence)-barcode_len]

                    #check if barcode is complete, otherwise drop
                    if not 'N' in barcode:

                        #check if barcode has been found already
                        if barcode in barcodes2:
                            barcodes2[barcode][0] += 1
                            barcodes2[barcode][1].append(subseq)
                        else:
                            barcodes2[barcode] = [1, [subseq]]
                    else:
                        dropped = dropped + 1
                        dropped_bar += 1
                        #dropped_lines.write(line+"\t"+"incomplete barcode\n")
                
                #if not found
                else:
                    #print(sequence)
                    dropped = dropped + 1
                    dropped_prime += 1
                    #dropped_lines.write(line+"\t"+"missing primers\n")

                #logging
                if linenum%lognum == 0:
                    print ("Read line "+str(linenum)+" at time: "+str(time.time()-starttime))
                    
            linenum = linenum + 1

    with open('data/'+x+'/barcodes0.json', 'w') as fp:
        json.dump(barcodes1, fp)
    with open('data/'+x+'/barcodes1.json', 'w') as fp:
        json.dump(barcodes2, fp)
        
    print (str(readno)+" total reads")
    print (str(len(barcodes1))+" barcodes1")
    print (str(len(barcodes2))+" barcodes2")
    print (str(dropped_prime)+" reads dropped because of missing primers")
    print (str(dropped_short)+" reads dropped because of short sequence")
    print (str(dropped_bar)+" reads dropped because of incomplete barcodes")
    print (str(dropped)+" reads dropped ("+ str(dropped * 100.0/readno) +"%) because of missing primers or incomplete barcodes\n")
    
    print ("completed reading "+x+" time="+str(time.time()-starttime))
    print()
    

processed line 0. time: 0.037688493728637695
processed line 1000000. time: 11.543131589889526
processed line 2000000. time: 22.985753536224365
processed line 3000000. time: 34.76365637779236
processed line 4000000. time: 46.399455070495605
processed line 5000000. time: 58.06118702888489
processed line 6000000. time: 69.82406759262085
processed line 7000000. time: 82.01861810684204
processed line 8000000. time: 93.0155348777771
processed line 9000000. time: 105.30243921279907
processed line 10000000. time: 116.26160311698914
processed line 11000000. time: 127.38427209854126
processed line 12000000. time: 140.01864981651306
processed line 13000000. time: 151.18846988677979
processed line 14000000. time: 162.59464240074158
processed line 15000000. time: 175.81974339485168
processed line 16000000. time: 187.25167393684387
processed line 17000000. time: 198.47560858726501
processed line 18000000. time: 212.58148980140686
processed line 19000000. time: 223.62500596046448
processed line 20000

processed line 165000000. time: 1957.513744354248
processed line 166000000. time: 1969.4985568523407
processed line 167000000. time: 1981.484176158905
processed line 168000000. time: 1993.1672849655151
processed line 169000000. time: 2004.7708320617676
processed line 170000000. time: 2016.5653841495514
processed line 171000000. time: 2028.180927991867
processed line 172000000. time: 2039.7493317127228
processed line 173000000. time: 2051.45352435112
processed line 174000000. time: 2063.0828251838684
processed line 175000000. time: 2074.72639131546
processed line 176000000. time: 2086.3305671215057
processed line 177000000. time: 2097.882367372513
processed line 178000000. time: 2109.58842587471
processed line 179000000. time: 2121.1656568050385
processed line 180000000. time: 2132.7549974918365
processed line 181000000. time: 2144.2424240112305
processed line 182000000. time: 2156.018175840378
processed line 183000000. time: 2167.733743429184
processed line 184000000. time: 2179.412406

processed line 35000000. time: 3972.7827146053314
processed line 36000000. time: 3985.2947998046875
processed line 37000000. time: 3999.218948364258
processed line 38000000. time: 4013.1003544330597
processed line 39000000. time: 4026.8601882457733
processed line 40000000. time: 4040.678309440613
processed line 41000000. time: 4054.397510290146
processed line 42000000. time: 4068.276205062866
processed line 43000000. time: 4082.004715204239
processed line 44000000. time: 4095.6910860538483
processed line 45000000. time: 4109.424572229385
processed line 46000000. time: 4122.7652888298035
processed line 47000000. time: 4135.887768983841
processed line 48000000. time: 4149.746890068054
processed line 49000000. time: 4163.447549104691
processed line 50000000. time: 4177.176958084106
processed line 51000000. time: 4190.923803329468
processed line 52000000. time: 4204.898358106613
processed line 53000000. time: 4218.6229927539825
processed line 54000000. time: 4232.435640096664
processed lin

processed line 201000000. time: 6324.034902572632
processed line 202000000. time: 6338.172096729279
processed line 203000000. time: 6352.3519904613495
processed line 204000000. time: 6366.3602640628815
processed line 205000000. time: 6380.476452827454
processed line 206000000. time: 6394.533830404282
processed line 207000000. time: 6408.615937709808
processed line 208000000. time: 6422.482609033585
processed line 209000000. time: 6436.606254577637
processed line 210000000. time: 6450.679815530777
processed line 211000000. time: 6464.804222345352
processed line 212000000. time: 6478.771696090698
processed line 213000000. time: 6492.759793043137
processed line 214000000. time: 6506.834668159485
processed line 215000000. time: 6520.773179292679
processed line 216000000. time: 6534.76694893837
processed line 217000000. time: 6548.70427942276
processed line 218000000. time: 6562.938215494156
processed line 219000000. time: 6576.8350393772125
processed line 220000000. time: 6591.124777078628

processed line 66000000. time: 815.5282294750214
processed line 67000000. time: 826.8213770389557
processed line 68000000. time: 837.8700745105743
processed line 69000000. time: 849.0584049224854
processed line 70000000. time: 860.1529958248138
processed line 71000000. time: 871.3053390979767
processed line 72000000. time: 882.3920912742615
processed line 73000000. time: 893.5794401168823
processed line 74000000. time: 904.6853885650635
processed line 75000000. time: 915.7351126670837
processed line 76000000. time: 926.8145434856415
processed line 77000000. time: 937.9283599853516
processed line 78000000. time: 949.1505351066589
processed line 79000000. time: 960.3222131729126
processed line 80000000. time: 971.1697635650635
processed line 81000000. time: 981.9921350479126
processed line 82000000. time: 993.2315540313721
processed line 83000000. time: 1004.3688724040985
processed line 84000000. time: 1025.1113934516907
processed line 85000000. time: 1036.1422345638275
processed line 86

...

processed line 218000000. time: 2599.825344324112
processed line 219000000. time: 2611.3726794719696
processed line 220000000. time: 2623.020991563797
processed line 221000000. time: 2634.61189866066
processed line 222000000. time: 2672.960362434387
processed line 223000000. time: 2684.3387694358826
processed line 224000000. time: 2695.7242591381073
processed line 225000000. time: 2707.2126293182373
processed line 226000000. time: 2718.694727897644
processed line 227000000. time: 2730.4363961219788
processed line 228000000. time: 2742.0064136981964
processed line 229000000. time: 2753.5737240314484
processed line 230000000. time: 2765.1105308532715
processed line 231000000. time: 2776.728428840637
processed line 232000000. time: 2788.324075460434
processed line 233000000. time: 2800.090010166168
processed line 234000000. time: 2811.666689157486
processed line 235000000. time: 2823.2153809070587
processed line 236000000. time: 2834.7842967510223
processed line 237000000. time: 2846.3238

processed line 89000000. time: 4723.093015432358
processed line 90000000. time: 4776.926899194717
processed line 91000000. time: 4790.647164106369
processed line 92000000. time: 4804.494318962097
processed line 93000000. time: 4818.485884666443
processed line 94000000. time: 4832.360426425934
processed line 95000000. time: 4846.286953687668
processed line 96000000. time: 4860.2378940582275
processed line 97000000. time: 4874.183421611786
processed line 98000000. time: 4888.1600041389465
processed line 99000000. time: 4902.003271818161
processed line 100000000. time: 4915.969869375229
processed line 101000000. time: 4929.8872056007385
processed line 102000000. time: 4943.8398904800415
processed line 103000000. time: 4957.90715098381
processed line 104000000. time: 4971.982675552368
processed line 105000000. time: 4986.122110366821
processed line 106000000. time: 5000.106088399887
processed line 107000000. time: 5014.044900417328
processed line 108000000. time: 5028.090795993805
processe

processed line 254000000. time: 7076.951400518417
processed line 255000000. time: 7091.0549919605255
processed line 256000000. time: 7105.039528369904
processed line 257000000. time: 7119.293590545654
processed line 258000000. time: 7133.522263765335
processed line 259000000. time: 7147.743990182877
processed line 260000000. time: 7161.868965387344
processed line 261000000. time: 7175.80028629303
processed line 262000000. time: 7190.128379106522
processed line 263000000. time: 7204.211706876755
processed line 264000000. time: 7218.275881767273
processed line 265000000. time: 7232.197286128998
processed line 266000000. time: 7245.482102870941
processed line 267000000. time: 7259.800808429718
processed line 268000000. time: 7274.093892097473
processed line 269000000. time: 7288.220632314682
processed line 270000000. time: 7302.223459720612
processed line 271000000. time: 7316.477211475372
processed line 272000000. time: 7330.658639431
processed line 273000000. time: 7344.753415107727
pro

processed line 119000000. time: 1458.5946235656738
processed line 120000000. time: 1470.0166804790497
processed line 121000000. time: 1481.6237320899963
processed line 122000000. time: 1493.0663626194
processed line 123000000. time: 1504.7281866073608
processed line 124000000. time: 1516.2497100830078
processed line 125000000. time: 1542.8850169181824
processed line 126000000. time: 1554.1511919498444
processed line 127000000. time: 1565.7587151527405
processed line 128000000. time: 1577.2525844573975
processed line 129000000. time: 1588.8817036151886
processed line 130000000. time: 1600.3976933956146
processed line 131000000. time: 1611.9140093326569
processed line 132000000. time: 1623.3893175125122
processed line 133000000. time: 1634.950140953064
processed line 134000000. time: 1646.5549862384796
processed line 135000000. time: 1658.3690292835236
processed line 136000000. time: 1669.9287555217743
processed line 137000000. time: 1682.287157535553
processed line 138000000. time: 1693

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [6]:
#b is consensus
def seqcompare(a,b):
	u=zip(a,b)
	x=[]
	for i in u: 
	    if i[0]==i[1] or i[1]=='N':
	        x.append('*') 
	    else: 
	        x.append(i[1])
	return "".join(x) 

In [ ]:
from collections import Counter
import time
import json

_MAJORITY_FACTOR = .8
_FAMILIES = 2

targets = ['GCAGTGAAAGAGTTCTTCGGTTCCAGCCAGCTGTCTCAGTTTATGGACCAGAACAACCCGCTGTCTGAGATTACGCACAAACGTCGTATCTCCGCACTCGGCCCAGGCGGTCTGACCCGTGAACGTGCA',
          'CGAAGTTCGAGACGTACACCCGACTCACTACGGTCGCGTATGTCCAATCGAAACCCCTGAAGGTCCGAACATCGGTCTGATCAACTCTCTGTCCGTGTACGCACAGACTAACGAATACGGCTTCCTTGAGACTCC']

for j, x in enumerate(samples):

    print ("Working on sample: " + x)
    
    for k, target in enumerate(targets):
    
        with open('data/'+x+'/barcodes'+str(k)+'.json', 'r') as fp:
            barcodes = json.load(fp)

        fp = open('data/'+x+'/barcodes_majority'+str(k)+'.txt', 'w')

        line = 0
        _LOG = 100000
        num_majority = 0

        starttime = time.time()
        filelen = len(barcodes)
        print ('Total barcodes: ' + str(filelen))

        for key, value in barcodes.items():

            if value[0] >= _FAMILIES:

                consensus = []
                zi = zip(*value[1])    

                for i, z in enumerate(zi):
                    best = Counter(z).most_common(1)[0]

                    if best[1] > len(z) * _MAJORITY_FACTOR:
                        consensus.append(best[0])
                    else:
                        consensus.append(target[i])

                consensus = "".join(consensus)

                snps = seqcompare(target, consensus)
                c = len(target) - snps.count('*')

                #fpu.write(str(value[0])+'\t'+key+'\t'+consensusu+'\t'+str(cu)+'\t'+snpsu+'\n')
                fp.write(str(value[0])+'\t'+key+'\t'+consensus+'\t'+str(c)+'\t'+snps+'\n')
                num_majority+=1

            if line%_LOG == 0:
                print ('Working on line ' + str(line) +' at time: '+str(time.time()-starttime)+'. '+str(float(line)*100/filelen)+'% complete.')
            line += 1

        print("Majority barcodes: " + str(num_majority))

        fp.close()

Working on sample: ky10
Total barcodes: 20319635
Working on line 0 at time: 0.0022165775299072266. 0.0% complete.
Working on line 100000 at time: 98.74407935142517. 0.49213482427218797% complete.
Working on line 200000 at time: 199.09477162361145. 0.9842696485443759% complete.
Working on line 300000 at time: 297.0557315349579. 1.476404472816564% complete.
Working on line 400000 at time: 393.6599519252777. 1.9685392970887519% complete.
Working on line 500000 at time: 492.992995262146. 2.46067412136094% complete.
Working on line 600000 at time: 590.2187695503235. 2.952808945633128% complete.
Working on line 700000 at time: 686.099766254425. 3.444943769905316% complete.
Working on line 800000 at time: 783.5700421333313. 3.9370785941775037% complete.
Working on line 900000 at time: 878.403918504715. 4.429213418449692% complete.
Working on line 1000000 at time: 974.3944370746613. 4.92134824272188% complete.
Working on line 1100000 at time: 1068.0474843978882. 5.4134830669940674% complete.
W